# Feature filtering based on Mutual Information for classification

In [ ]:
import pandas as pd
import os
import numpy as np

# depending on the OS the path to the data file is different
if os.name == 'nt':
    data = pd.read_excel(r'..\data\nutrient-file-release2-jan22.xlsx', sheet_name='All solids & liquids per 100g')
elif os.name == 'posix':
    data = pd.read_excel(r'../data/nutrient-file-release2-jan22.xlsx', sheet_name='All solids & liquids per 100g')

# print the first 5 rows of the data
data.head()

Since the dataset is largely continuous, we need to turn features into discrete ones.

## Variable discretisation

There are different methods to discretise continuous variables:

- **Equal-width binning**: divides the scope of possible values into N bins of the same width.
- **Equal-frequency binning**: divides the scope of possible values into N bins, each of them containing approximately the same number of samples.
- **Domain knowledge binning**: divides the scope of possible values into bins according to the domain knowledge.
<!-- There are other methods too listed below.

- ChiMerge: merges the bins using the Chi2 test to evaluate the statistical dependence of the classes and the feature.
- Entropy-based binning: merges the bins using the entropy of the classes and the feature.
- K-means binning: merges the bins using the K-means algorithm.
- Gaussian mixture binning: merges the bins using a Gaussian Mixture Model.
- Quantile binning: merges the bins so that each bin contains the same number of samples.
- Uniform binning: merges the bins so that each bin contains the same width.
- Recursive partitioning: merges the bins using a decision tree.
- Discretisation using decision trees: merges the bins using a decision tree.
- Discretisation using clustering: merges the bins using a clustering algorithm.
- Discretisation using support vector machines: merges the bins using a support vector machine.
- Discretisation using linear models: merges the bins using a linear model.
- Discretisation using nearest neighbours: merges the bins using a nearest neighbours algorithm.
- Discretisation using kernel density estimation: merges the bins using a kernel density estimation.
- Discretisation using fuzzy logic: merges the bins using a fuzzy logic algorithm.
- Discretisation using genetic algorithms: merges the bins using a genetic algorithm.
- Discretisation using simulated annealing: merges the bins using a simulated annealing algorithm.
- Discretisation using a neural network: merges the bins using a neural network.
- Discretisation using a random forest: merges the bins using a random forest.
- Discretisation using a linear discriminant analysis: merges the bins using a linear discriminant analysis.
- Discretisation using a quadratic discriminant analysis: merges the bins using a quadratic discriminant analysis.
- Discretisation using a principal component analysis: merges the bins using a principal component analysis.
- Discretisation using a factor analysis: merges the bins using a factor analysis.
- Discretisation using a canonical correlation analysis: merges the bins using a canonical correlation analysis.
- Discretisation using a partial least squares regression: merges the bins using a partial least squares regression.
- Discretisation using a ridge regression: merges the bins using a ridge regression. -->

WARNING: The choice of bins will influence the results of the mutual information filter.

In [ ]:
# variable discretisation using pandas.qcut

# add new column with discretised values
data['Discretised Energy with dietary fibre, equated \n(kJ)'] = pd.qcut(data['Energy with dietary fibre, equated \n(kJ)'], 20, labels=False)

# print the first few rows of the data for the two columns
data[['Energy with dietary fibre, equated \n(kJ)', 'Discretised Energy with dietary fibre, equated \n(kJ)']].head(10)

# print the first few rows of the sorted data for the two columns
# data[['Energy with dietary fibre, equated \n(kJ)', 'Discretised Energy with dietary fibre, equated \n(kJ)']].sort_values(by='Energy with dietary fibre, equated \n(kJ)', ascending=False).head(10)